# <center>Блок 4. Введение в машинное обучение<center>
## <center>ML-3. Обучение с учителем: классификация<center>
### <center>1.Введение<center>
### <center>2.Логистическая регрессия<center>
#### <center>Общее представление о логистической регрессии<center>

In [17]:
# Задание 2.5
z = 1 + 5*0.3 - 0.3*3
p = 1 / (1+2.718**-z)
print(round(p,2))

0.83


#### <center>Поиск параметров логистической регрессии<center>